# Application Example: Gyroscope Allan Variance

The signals of, e.g., angular rate sensors contain white noise along with colored noise resulting from a variety of sources. With the ``pyplnoise``module, this noise can be generated and added on top of the deterministic sensor errors in a sensor model. Such a sensor model can be used for example for the simulation of a Kalman filter accommodating correlated noise terms.

In the code below it is outlined how you can use ``pyplnoise`` to approximate the Allan variance of a real yaw rate sensor. Familiarity with the IEEE Standard 952 is helpful.

In [1]:
import pyplnoise
import allan
import time
import numpy as np
import matplotlib.pyplot as plt
from bokeh.plotting import figure, output_file, show
from bokeh.io import output_notebook
from bokeh.models import Range1d
output_notebook()

Loading BokehJS ...

The angular rate sensor is described by the quantities $N$ (angle random walk), $B$ (bias instability) and $K$ (rate random walk) from the IEEE Std 952. The values prescribed here approximate the properties of a Bosch SMI130 yaw rate sensor.

In [2]:
N = 5e-3 # °/sqrt(s) = 0.3 °/sqrt(h)
B = 7e-4 # °/s = 2.5 °/h
K = 6e-6 # °/s/sqrt(s)

Define the sampling rate ``fs`` and initialize noise sources:

In [3]:
fs = 10.
npts = 60000000
t = np.linspace(0., npts/fs, num=npts)

In [4]:
whnoise = pyplnoise.WhiteNoise(fs)
rdnoise = pyplnoise.RedNoise(fs, 1e-8)
pknoise = pyplnoise.PinkNoise(fs, 1e-6, fs/2.)

Now create an artifical sensor noise signal $\omega_{noise}$ from the individual noise sources denoted by $\xi$. The noise is additive and the coefficients $N$, $B$ and $K$ from the IEEE Std 952 enter as follows:

$$\omega_{noise} = N\xi_{white} + \frac{B}{\sqrt{2\pi}}\xi_{pink} + \frac{K}{2\pi}\xi_{red}$$

In [5]:
w = N * whnoise.get_series(npts) + B / np.sqrt(2. * np.pi) * pknoise.get_series(npts) +\
K / (2. * np.pi) * rdnoise.get_series(npts)

Now calculate the Allan variance. The module ``allan`` is not provided by ``pyplnoise``, but many similar codes can be found on pypi.

In [6]:
tau, avar, adev, error_adev_pct = allan.allan_variance(w, dt=1./fs, n_clusters=8000, verbose=False)

In [7]:
# omit values with insufficient statistical quality
idx_valid = error_adev_pct < 10
avar_valid = avar[idx_valid]
adev_valid = adev[idx_valid]
tau_valid = tau[idx_valid]

Here's how the calculated Allan deviation looks like.

In [8]:
fig = figure(title="Allan deviation",
             x_axis_label="Cluster length (s)",
             y_axis_label="Allan deviation (°/h)",
             x_axis_type="log",
             y_axis_type="log")
fig.line(tau_valid, adev_valid*3600, line_width=1)
show(fig)

Let's extract the properties $N$, $B$ and $K$ automatically:

In [9]:
noise_density, a_n_fit, b_n_fit = allan.read_noise_density(adev_valid, tau_valid)
print("Extracted angle random walk: N = {:.4g} °/s/sqrt(Hz) = {:.5f} °/sqrt(h)"
      .format(noise_density, noise_density*60))
bias_instability = allan.read_bias_instability(adev_valid, tau_valid)
print("Extracted bias instability: B = {:.4g} °/s = {:.5f} °/h"
      .format(bias_instability, bias_instability*3600))
rate_random_walk, a_k_fit, b_k_fit = allan.read_rate_random_walk(adev_valid, tau_valid)
print("Extracted rate random walk: K = {:.4g} °/s^(3/2) = {:.5f} °/h^(3/2)"
      .format(rate_random_walk, rate_random_walk*3600**(3/2)))

Extracted angle random walk: N = 0.005026 °/s/sqrt(Hz) = 0.30156 °/sqrt(h)
Extracted bias instability: B = 0.0008001 °/s = 2.88031 °/h
Extracted rate random walk: K = 6.676e-06 °/s^(3/2) = 1.44207 °/h^(3/2)


If you rather prefer manual extraction, you can read off the value of the red line at $\tau = 1$ to get $N$. The value of the green line at $\tau = 3$ is equal to $K$, and multiplication of the global minimum by 0.664 yields $B$.

In [10]:
fit_vals_n = np.power(10.0, a_n_fit * np.log10(tau_valid) + b_n_fit)
fit_vals_k = np.power(10.0, a_k_fit * np.log10(tau_valid) + b_k_fit)
fig = figure(title="Allan deviation",
             x_axis_label="Cluster length (s)",
             y_axis_label="Allan deviation (°/s)",
             x_axis_type="log",
             y_axis_type="log")
fig.line(tau_valid, adev_valid, line_width=1)
fig.line(tau_valid, fit_vals_n, line_width=1, line_color="red")
fig.line(tau_valid, fit_vals_k, line_width=1, line_color="green")
show(fig)